# Training

## Variables

In [ ]:
ROBOFLOW_API_KEY = "YOUR_API_KEY"
ROBOFLOW_WORKSPACE = "scorpionsvision"
ROBOFLOW_PROJECT = "conev3"
ROBOFLOW_PROJECT_VERSION = 1
BATCH_SIZE = -1  # -1 for autobatch
IMAGE_SIZE = 640
EPOCHS = 20
DETECTION_THRESHOLD = 0.01

## Torch Installation

In [ ]:
!pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html # newer versions of pytorch are not compatible
!nvidia-smi
!nvcc --version
!python -c "import torch; print(torch.version.cuda)"

## Yolov5-OBB Installation

In [ ]:
!git clone https://github.com/hukaixuan19970627/yolov5_obb.git

In [ ]:
%cd /content/yolov5_obb/
!pip install -r requirements.txt

In [ ]:
%cd /content/yolov5_obb/utils/nms_rotated
!python setup.py develop

## Data Preparation

In [ ]:
!mkdir /content/datasets/
%cd /content/datasets/

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
dataset = project.version(ROBOFLOW_PROJECT_VERSION).download("yolov5-obb")

In [ ]:
!mv /content/datasets/{ROBOFLOW_PROJECT}-{ROBOFLOW_PROJECT_VERSION}/ /content/datasets/roboflow/

## Train

In [ ]:
%cd /content/yolov5_obb/
!python train.py --weights weights/yolov5s --data /content/datasets/roboflow/data.yaml --epochs {EPOCHS} --batch-size {BATCH_SIZE} --img {IMAGE_SIZE} --device 0 --exist-ok

# Detection/Inference

## Detect

In [ ]:
%cd /content/yolov5_obb/
!python detect.py --weights 'runs/train/exp/weights/best.pt' --source '../datasets/roboflow/valid/images' --conf-thres {DETECTION_THRESHOLD} --img {IMAGE_SIZE} --device 0 --agnostic-nms

## Show a random result

In [ ]:
import os, random
random_file = random.choice(os.listdir("/content/yolov5_obb/runs/detect/exp"))

from IPython.display import Image
Image(random_file)

# Utils

In [ ]:
# Remove detect folder
!rm -rf /content/yolov5_obb/runs/detect/

In [ ]:
# Remove train folder
!rm -rf /content/yolov5_obb/runs/train/